In [ ]:
import math
import numpy as np

In [1]:
class Actuator:
    def __init__(self, initial_torque):
        self.torque = initial_torque

    def apply_torque(self, torque):
        self.torque = torque
        # Implement the logic to apply torque to the joint


class Sensor:
    def __init__(self):
        self.position = 0
        self.velocity = 0
        self.acceleration = 0

    def update_readings(self, position, velocity, acceleration):
        self.position = position
        self.velocity = velocity
        self.acceleration = acceleration
        # Update sensor readings based on the joint's movement


class Joint:
    def __init__(self, actuator, sensor, initial_degree):
        self.actuator = actuator
        self.sensor = sensor
        self.degree = initial_degree

    def move(self, torque):
        # Use the actuator to apply torque
        self.actuator.apply_torque(torque)
        # Logic for moving the joint and updating the sensor readings
        # For example:
        self.sensor.update_readings(self.degree, 0, 0)  # Placeholder values


class RoboticArm:
    def __init__(self):
        self.joint1 = Joint(Actuator(initial_torque=0), Sensor(), initial_degree=0)
        self.joint2 = Joint(Actuator(initial_torque=0), Sensor(), initial_degree=0)
        self.position_sensor = Sensor()

    def move_joints(self, torque1, torque2):
        self.joint1.move(torque1)
        self.joint2.move(torque2)
        # Logic to determine the end position based on joint movements
        # Update position sensor
        self.position_sensor.update_readings(0, 0, 0)  # Placeholder values

    def get_end_position(self):
        # Calculate the end position of the robotic arm
        # Return end position
        return (0, 0)  # Placeholder values


# Example usage:
robotic_arm = RoboticArm()
robotic_arm.move_joints(torque1=10, torque2=20)
end_position = robotic_arm.get_end_position()
print(f"The end position of the robotic arm is {end_position}")

The end position of the robotic arm is (0, 0)


In [3]:
# Define the parameters and state variables
m1, m2 = 0.8, 0.4  # masses of the links
l1, l2 = 1.0, 0.5  # lengths of the links
g = 9.81  # acceleration due to gravity
theta1, theta2 = np.radians(30), np.radians(45)  # joint angles in radians
dtheta1, dtheta2 = 0.0, 0.0  # joint angular velocities
ddtheta1, ddtheta2 = 0.0, 0.0  # joint angular accelerations

# Define the equations using numpy for trigonometric functions and linear algebra
def calculate_torques(theta1, theta2, dtheta1, dtheta2, ddtheta1, ddtheta2):
    D11 = (m1 + m2) * l1**2 + m2 * l2**2 + 2 * m2 * l1 * l2 * np.cos(theta2)
    D12 = D21 = m2 * l2**2 + m2 * l1 * l2 * np.cos(theta2)
    D22 = m2 * l2**2

    C1 = -m2 * l1 * l2 * np.sin(theta2) * dtheta2**2 - m2 * l1 * l2 * np.sin(theta2) * (dtheta1 + dtheta2)
    C2 = m2 * l1 * l2 * np.sin(theta2) * dtheta1**2

    G1 = (m1 + m2) * g * l1 * np.cos(theta1) + m2 * g * l2 * np.cos(theta1 + theta2)
    G2 = m2 * g * l2 * np.cos(theta1 + theta2)

    # Inertia matrix
    D = np.array([[D11, D12],
                  [D21, D22]])
    
    # Coriolis and centrifugal vector
    C = np.array([C1, C2])
    
    # Gravity vector
    G = np.array([G1, G2])
    
    # Angular acceleration vector
    ddtheta = np.array([ddtheta1, ddtheta2])
    
    # Calculate torques
    T = np.dot(D, ddtheta) + C + G
    return T

# Calculate torques for the given state
torques = calculate_torques(theta1, theta2, dtheta1, dtheta2, ddtheta1, ddtheta2)
torques

array([10.70265402,  0.50780297])

In [ ]:
def control(theta1, d_theta1, DD_theta1, theta2, d_theta2, DD_theta2, lambda1, de1, lambda2, de2):
    # Constants
    l1 = 1; m1 = 0.8; l2 = 0.5; m2 = 0.4; g = 9.8
    
    # Inertia matrix components
    D11 = (m1 + m2) * l1**2 + m2 * l2**2 + 2 * m2 * l1 * l2 * np.cos(theta2)
    D12 = m2 * l2**2 + m2 * l1 * l2 * np.cos(theta2)
    D21 = D12  # since D12 == D21 in a symmetric matrix
    D22 = m2 * l2**2
    
    # Coriolis and centrifugal force vector components
    Z = m2 * l1 * l2 * np.sin(theta2)
    
    # Gravity vector components
    G1 = (m1 + m2) * g * l1 * np.cos(theta1) + m2 * g * l2 * np.cos(theta1 + theta2)
    G2 = m2 * g * l2 * np.cos(theta1 + theta2)
    
    # Torque inputs
    T1 = D11 * DD_theta1 + D12 * DD_theta2 - Z * d_theta2**2 + G1
    T2 = D21 * DD_theta1 + D22 * DD_theta2 + Z * d_theta1**2 + G2
    
    # Control outputs
    y1 = -T1 + DD_theta1 - lambda1 * de1
    y2 = -T2 + DD_theta2 - lambda2 * de2
    
    return y1, y2

# Example usage:
theta1, d_theta1, DD_theta1 = np.radians(30), 0, 0
theta2, d_theta2, DD_theta2 = np.radians(45), 0, 0
lambda1, de1 = 5, 0.1
lambda2, de2 = 5, 0.1

y1, y2 = control(theta1, d_theta1, DD_theta1, theta2, d_theta2, DD_theta2, lambda1, de1, lambda2, de2)
print(f"Control Output 1: {y1}, Control Output 2: {y2}")

In [ ]:
def smc_signal(e, d_e, K, lambda_, delay):
    s = lambda_ * e + d_e
    if abs(s) > delay:
        sat_s = np.sign(s)
    else:
        sat_s = s / delay
    SMC_Signal = -K * sat_s
    return SMC_Signal

# Example usage:
e = 0.1  # error
d_e = 0.01  # derivative of error
K = 5  # gain
lambda_ = 2  # lambda
delay = 0.01  # delay

signal = smc_signal(e, d_e, K, lambda_, delay)
print(f"SMC Signal: {signal}")

In [ ]:
t = np.linspace(0, 10, 500)  # 10 seconds, 500 points

# Sinusoidal input for desired x value
amplitude = 1  # Amplitude of the sine wave
frequency = 1  # Frequency (number of cycles per second)
desired_x = amplitude * np.sin(2 * np.pi * frequency * t)